In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import optuna
from functools import partial

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_folder = '~/Data/Kaggle/Titanic/'

In [5]:
train_file = file_folder + 'train_processed.csv'

In [6]:
df = pd.read_csv(train_file)

In [7]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,0,1,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,0,1,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,0,1,0,1,0,0,1


In [9]:
y = df['Survived'].values
x = df.drop(['Survived'], axis=1).values

# Build Random Forest model

In [ ]:
def optimize(trial, x, y):
    entropy = trial.suggest_categorical("criterion", ['gini', 'entropy'])
    n_estimators = trial.suggest_int("n_estimators", 100, 500)
    max_depth = trial.suggest_int("max_depth", 3, 6)
    max_features = trial.suggest_uniform("max_features", 0.01, 1.0)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=entropy,
        n_jobs=-1
    )

    kf = model_selection.StratifiedKFold(n_splits=5)

    accuracies = []
    for idx in kf.split(X=x, y=y):
        train_idx, test_idx = idx[0], idx[1]

        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        model.fit(xtrain, ytrain)
        preds = model.predict(xtest)
        fold_acc = metrics.accuracy_score(ytest, preds)
        accuracies.append(fold_acc)

    return -1.0*np.mean(accuracies)

In [ ]:
optimization_function = partial(optimize, x=x, y=y)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(optimization_function, n_trials=15)